# Ensembles

On aimerait bien pouvoir illustrer la théorie naïve des ensembles en python... Au moins jusqu'à l'ensemble des parties d'un ensemble. 

## Modélisation

### La classe set

In [25]:
# on peut faire un ensemble
a = set([1,2,3])
print("a =",a)

# même un deuxième ensemble
b = set(['A', 'B'])
print("b =",b)

a = {1, 2, 3}
b = {'A', 'B'}


In [27]:
# mais si on cherche à faire un ensemble d'ensembles...
c = set([a,b])

TypeError: unhashable type: 'set'

On constate qu'un objet doit être "hashable" pour pouvoir être élément d'un "set", ce qu'un objet de type "set" n'est pas.

### La classe frozenset à la rescousse

La classe "frozenset" dispose des mêmes méthodes que la classe "set" tout en étant "hashable". En outre un "frozenset" est immutable.

In [1]:
# on peut faire un ensemble à l'aide d'un frozenset
a = frozenset([1,2,3])
print("a =", a)

# et même un deuxième
b = frozenset(['A', 'B'])
print("b =", b)

# et même un ensemble d'ensembles !
c = frozenset([a,b])
print("c =", c)

a = frozenset({1, 2, 3})
b = frozenset({'A', 'B'})
c = frozenset({frozenset({1, 2, 3}), frozenset({'A', 'B'})})


On peut faire des ensembles d'ensembles ! Mais l'affichage laisse à désirer...

### La classe Ensemble

In [4]:
# étendons la classe frozenset
class Ensemble(frozenset):
    # en réécrire l'affichage
    def __repr__(self):
        return "{" + ", ".join(map(str,self)) + "}" # un frozenset est itérable
    
a = Ensemble([1,2])
b = Ensemble(['a', 'b'])
print("a =",a)
print("b =",b)

a = {1, 2}
b = {b, a}


La représentation d'un ensemble est la même qu'en mathématiques désormais. Simplement, on aimerait ne pas avoir à passer une liste en paramètre du constructeur, mais directement les éléments.

In [9]:
class Ensemble(frozenset):
    # __new__ construit, puis __init__ initialise
    def __new__(cls, *elements):
        # elements est la liste des arguments positionnels !
        return super().__new__(cls, elements)
    def __repr__(self):
        return "{" + ", ".join(map(str,self)) + "}"

# construisons des ensembles sans les crochets
a = Ensemble(1,2,3)
b = Ensemble('a', 'b')
c = Ensemble(a,b,Ensemble())
print("a =",a)
print("b =",b)
print("c =", c)

a = {1, 2, 3}
b = {b, a}
c = {{1, 2, 3}, {b, a}, {}}


### Quelques "méthodes magiques"

On aimerait pouvoir utiliser les opérations ensemblistes classiques. Ce qui est possible mais...

In [13]:
print(a.union(b))

frozenset({1, 2, 3, 'b', 'a'})


On obtient une réunion de type "frozenset" :( . Remédions à cela.

In [26]:
class Ensemble(frozenset):
    def __new__(cls, *elements):
        return super().__new__(cls, elements)
    def __repr__(self):
        return "{" + ", ".join(map(str,self)) + "}"
    
    # réécrivons l'union de deux ensembles
    def union(self, autre):
        return Ensemble(*frozenset.union(self, autre))
    def __or__(self, autre):
        return self.union(autre)
    
a = Ensemble(*"Bonjour!")
b = Ensemble("Toto")
print("a.union(b) =", a.union(b))
print("a|b =", a|b)

a.union(b) = {j, r, n, u, B, o, Toto, !}
a|b = {j, r, n, u, B, o, Toto, !}


## L'ensemble des parties d'un ensemble

In [23]:
class Ensemble(frozenset):
    def __new__(cls, *elements):
        return super().__new__(cls, elements)
    def __repr__(self):
        return "{" + ", ".join(map(str,self)) + "}"
    def union(self, autre):
        return Ensemble(*frozenset.union(self, autre))
    def __or__(self, autre):
        return self.union(autre)
    # une méthode itérative pour l'ensemble des parties d'un ensemble
    def parties(self):
        ens = list(self) # on veut pouvoir utiliser les [] (subscriptable)
        parties = []
        n = len(self)
        for i in range(2**n):
            partie = [] 
            for j in range(n):
                # le j-eme bit indique la présence dans la partie du j-eme élément
                if (1 << j) & i :
                    partie.append(ens[j])
            parties.append(Ensemble(*partie))
        return Ensemble(*parties)

a = Ensemble(20, 30, 42)
print(a.parties())
print(Ensemble().parties().parties().parties())

{{20, 30}, {42, 20}, {42, 30}, {20}, {30}, {42}, {}, {42, 20, 30}}
{{}, {{}}, {{}, {{}}}, {{{}}}}


## Produit cartésien

In [28]:
import ensembles.ensembles as ens

a = ens.Ensemble(5)
b = ens.Ensemble(20, 'A')

print((a*b).parties())

[Ø, {(5, 20)}, {(5, 'A')}, {(5, 20), (5, 'A')}]
{Ø, {(5, 20)}, {(5, 'A')}, {(5, 20), (5, 'A')}}
